In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import numpy as np
import torch.optim as optim
from tqdm import tqdm
import sys, os, math
import wandb
import warnings
warnings.simplefilter("ignore")


sys.path.insert(0, '../dlp')
from batch import Batch

device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
print(device)

epochs = 100_000
val_epoch = 100
num_val = 50
batch_size = 64
dataset_name = "corpus_200_500_random"
lr = 0.01
model_name = "CNN last level"
max_seq_len = 500

from data_access import PQDataAccess
da = PQDataAccess(f"/home/aac/Alireza/datasets/export_pqt_4_taxseq_new/{dataset_name}", batch_size)

checkpoint_dir = f"../checkpoints/{model_name}_checkpoints"
if not os.path.exists(checkpoint_dir):
    os.makedirs(checkpoint_dir)
print(checkpoint_dir)

wandb.init(
    # set the wandb project where this run will be logged
    project=model_name,

    # track hyperparameters and run metadata
    config={
        "learning_rate": lr,
        "architecture": "FNN",
        "dataset": dataset_name,
        "epochs": epochs,
        "batch_szie": batch_size,
        "max_seq_len": max_seq_len
    }
)

cuda:1
 WORLD_SIZE=1 , LOCAL_WORLD_SIZE=1,RANK =0,LOCAL_RANK = 0 
../checkpoints/FNN last level_checkpoints


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: alirezanor (alireza_noroozi). Use `wandb login --relogin` to force relogin


In [ ]:
df = pd.read_csv('../data/new_output_id.csv')

unique_labels = sorted(set(df['new label'].values))
label_to_idx = {label: idx for idx, label in enumerate(unique_labels)}
idx_to_label = {idx: label for label, idx in label_to_idx.items()}
map_label = {r['Organism_ID']: label_to_idx[r['new label']] for _, r in df.iterrows()}

print("# of Classes", len(unique_labels))
num_classes = len(unique_labels)

vocab = "ACDEFGHIKLMNPQRSTVWY"
char_to_idx = {char: idx + 1 for idx, char in enumerate(vocab)}  # Start index from 1 for padding
# Sequence encoder: Convert the protein sequence into integers
def encode_sequence(sequence):
    return [char_to_idx.get(char, 0) for char in sequence] + [0 for _ in range(max_seq_len - len(sequence))]  # 0 for unknown characters or padding 


def data_to_tensor_batch(b):
    inputs = [encode_sequence(e['Sequence']) for e in b]
    tax_ids = [map_label[e['Taxonomic_lineage_IDs']] for e in b]
    return Batch(torch.LongTensor(inputs), torch.LongTensor(tax_ids))

In [13]:
class FNN(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.embedding = nn.Embedding(21, 1280)
        self.attention = nn.Sequential(
            nn.Linear(1280, 256),
            nn.Tanh(),
            nn.Linear(256, 1)
        )
        
        self.layer1 = nn.Linear(1280, 512)
        self.layer_norm = nn.LayerNorm(512)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.1)
        self.layer2 = nn.Linear(512, num_classes)
        
    def attention_pooling(self, x):
        # x shape: (batch_size, seq_length, embedding_dim)
        attention_weights = self.attention(x)  # (batch_size, seq_length, 1)
        attention_weights = torch.softmax(attention_weights.squeeze(-1), dim=1)  # (batch_size, seq_length)
        attention_weights = attention_weights.unsqueeze(-1)  # (batch_size, seq_length, 1)
        pooled = torch.sum(x * attention_weights, dim=1)  # (batch_size, embedding_dim)
        return pooled
        
    def forward(self, x):
        outputs = self.embedding(x)
        
        outputs = self.attention_pooling(outputs)
        
        outputs = self.layer1(outputs)
        outputs = self.layer_norm(outputs)
        outputs = self.relu(outputs)
        outputs = self.dropout(outputs)
        outputs = self.layer2(outputs)
        return outputs

In [14]:
model = FNN(num_classes).to(device)

total = sum(p.numel() for p in model.parameters())
print(f'Total parameters: {total/ 1e6} M')
print(model)

optimizer = torch.optim.Adam(model.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()
# Cosine annealing with warm restarts
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(
    optimizer,
    T_0=10,  # Initial restart interval
    T_mult=2,  # Multiply interval by 2 after each restart
    eta_min=1e-6  # Minimum learning rate
)

Total parameters: 27.663345 M
FNN(
  (embedding): Embedding(21, 1280)
  (attention): Sequential(
    (0): Linear(in_features=1280, out_features=256, bias=True)
    (1): Tanh()
    (2): Linear(in_features=256, out_features=1, bias=True)
  )
  (layer1): Linear(in_features=1280, out_features=512, bias=True)
  (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
  (relu): ReLU()
  (dropout): Dropout(p=0.1, inplace=False)
  (layer2): Linear(in_features=512, out_features=51952, bias=True)
)


In [15]:
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, precision_score, recall_score

def compute_metrics(all_labels, all_preds, running_loss, num_val):
    # Convert tensors to numpy arrays
    labels_np = torch.cat(all_labels).numpy()
    preds_np = torch.cat(all_preds).numpy()

    # print("labels:", labels_np)
    # print("preds:", preds_np)
    
    # Compute evaluation metrics
    accuracy = accuracy_score(labels_np, preds_np)
    
    # Precision scores
    precision_macro = precision_score(labels_np, preds_np, average='macro')
    precision_micro = precision_score(labels_np, preds_np, average='micro')
    
    # Recall scores
    recall_macro = recall_score(labels_np, preds_np, average='macro')
    recall_micro = recall_score(labels_np, preds_np, average='micro')
    
    # F1 scores
    f1_macro = f1_score(labels_np, preds_np, average='macro')
    f1_micro = f1_score(labels_np, preds_np, average='micro')
    
    # Confusion matrix
    # conf_matrix = confusion_matrix(labels_np, preds_np, labels=[_ for _ in range(num_classes)])
    
    # Average loss
    avg_loss = running_loss / num_val
    
    return {
        'loss': avg_loss,
        'accuracy': accuracy,
        'precision_macro': precision_macro,
        'precision_micro': precision_micro,
        'recall_macro': recall_macro,
        'recall_micro': recall_micro,
        'f1_macro': f1_macro,
        'f1_micro': f1_micro,
        # 'confusion_matrix': conf_matrix
    }

In [16]:
val_batches = [da.get_batch() for _ in range(num_val)]

def evaluate():
    model.eval()  # Set model to evaluation mode
    
    running_loss = 0.0
    all_preds = []
    all_labels = []
    
    for epoch in range(num_val):
        with torch.no_grad():  # Disable gradient computation during evaluation
            tensor_batch = data_to_tensor_batch(val_batches[epoch])
            tensor_batch.gpu(device)
        
            labels = tensor_batch.taxes
            outputs = model(tensor_batch.seq_ids)

            # Calculate the loss
            loss = criterion(outputs, labels)
    
            running_loss += loss.item()
                
            preds = torch.argmax(outputs, dim=1)
    
            all_preds.append(preds.cpu())
            all_labels.append(labels.cpu())
    
    return compute_metrics(all_labels, all_preds, running_loss, num_val)

In [18]:
running_loss = 0
train_preds = []
train_labels = []
current_lr = lr

for epoch in tqdm(range(epochs)):
    model.train()

    tensor_batch = data_to_tensor_batch(da.get_batch())
    tensor_batch.gpu(device)
    
    labels = tensor_batch.taxes
    outputs = model(tensor_batch.seq_ids)
    loss = criterion(outputs, labels)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    running_loss += loss.item()
    
    preds = torch.argmax(outputs, dim=1)
    
    train_preds.append(preds.cpu())
    train_labels.append(labels.cpu())
    
    if (epoch + 1) % val_epoch == 0:
        # Calculate training metrics
        train_metrics = compute_metrics(train_labels, train_preds, running_loss, val_epoch)

        print(f"Epoch [{epoch + 1}/{epochs}]")
        # Training metrics
        print(f"Train Loss: {train_metrics['loss']:.4f}, Train Accuracy: {train_metrics['accuracy']:.4f}")
        print(f"Train Precision (micro): {train_metrics['precision_micro']:.4f}, Train Precision (macro): {train_metrics['precision_macro']:.4f}")
        print(f"Train Recall (micro): {train_metrics['recall_micro']:.4f}, Train Recall (macro): {train_metrics['recall_macro']:.4f}")
        print(f"Train F1 (micro): {train_metrics['f1_micro']:.4f}, Train F1 (macro): {train_metrics['f1_macro']:.4f}")
        # print("Train Confusion Matrix:")
        # print(train_metrics['confusion_matrix'])

        print()
        val_metrics = evaluate()
        
        # Validation metrics
        print(f"Val Loss: {val_metrics['loss']:.4f}, Val Accuracy: {val_metrics['accuracy']:.4f}")
        print(f"Val Precision (micro): {val_metrics['precision_micro']:.4f}, Val Precision (macro): {val_metrics['precision_macro']:.4f}")
        print(f"Val Recall (micro): {val_metrics['recall_micro']:.4f}, Val Recall (macro): {val_metrics['recall_macro']:.4f}")
        print(f"Val F1 (micro): {val_metrics['f1_micro']:.4f}, Val F1 (macro): {val_metrics['f1_macro']:.4f}")
        # print("Val Confusion Matrix:")
        # print(val_metrics['confusion_matrix'])
        print("-" * 80)  # Separator
        
        metrics = {
            # Training metrics
            "train_loss": train_metrics['loss'],
            "train_accuracy": train_metrics['accuracy'],
            "train_precision_micro": train_metrics['precision_micro'],
            "train_precision_macro": train_metrics['precision_macro'],
            "train_recall_micro": train_metrics['recall_micro'],
            "train_recall_macro": train_metrics['recall_macro'],
            "train_f1_micro": train_metrics['f1_micro'],
            "train_f1_macro": train_metrics['f1_macro'],
            # "train_confusion_matrix": train_metrics['confusion_matrix'],
            
            # Validation metrics
            "val_loss": val_metrics['loss'],
            "val_accuracy": val_metrics['accuracy'],
            "val_precision_micro": val_metrics['precision_micro'],
            "val_precision_macro": val_metrics['precision_macro'],
            "val_recall_micro": val_metrics['recall_micro'],
            "val_recall_macro": val_metrics['recall_macro'],
            "val_f1_micro": val_metrics['f1_micro'],
            "val_f1_macro": val_metrics['f1_macro'],
            # "val_confusion_matrix": val_metrics['confusion_matrix'],
            
            # Additional info
            "epoch": epoch + 1,
            "lr": current_lr
        }

        # Save periodic checkpoint
        checkpoint_path = os.path.join(checkpoint_dir, f'checkpoint_epoch_{epoch}.pt')
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'scheduler_state_dict': scheduler.state_dict(),
            'metrics': metrics
        }, checkpoint_path)
        
        # Log to wandb
        wandb.log(metrics)

        # Step the scheduler
        scheduler.step(epoch + loss.item())
        current_lr = scheduler.get_last_lr()[0]
        
        # Reset training metrics
        running_loss = 0
        train_preds = []
        train_labels = []

wandb.finish()

  0%|          | 95/100000 [00:02<37:08, 44.83it/s]

Epoch [100/100000]
Train Loss: 7.0688, Train Accuracy: 0.0353
Train Precision (micro): 0.0353, Train Precision (macro): 0.0001
Train Recall (micro): 0.0353, Train Recall (macro): 0.0010
Train F1 (micro): 0.0353, Train F1 (macro): 0.0001

Val Loss: 7.1143, Val Accuracy: 0.0384
Val Precision (micro): 0.0384, Val Precision (macro): 0.0001
Val Recall (micro): 0.0384, Val Recall (macro): 0.0018
Val F1 (micro): 0.0384, Val F1 (macro): 0.0002
--------------------------------------------------------------------------------


  0%|          | 195/100000 [00:05<33:48, 49.20it/s]  

Epoch [200/100000]
Train Loss: 7.0917, Train Accuracy: 0.0364
Train Precision (micro): 0.0364, Train Precision (macro): 0.0001
Train Recall (micro): 0.0364, Train Recall (macro): 0.0012
Train F1 (micro): 0.0364, Train F1 (macro): 0.0001

Val Loss: 7.0807, Val Accuracy: 0.0347
Val Precision (micro): 0.0347, Val Precision (macro): 0.0001
Val Recall (micro): 0.0347, Val Recall (macro): 0.0015
Val F1 (micro): 0.0347, Val F1 (macro): 0.0002
--------------------------------------------------------------------------------


  0%|          | 298/100000 [00:09<35:55, 46.26it/s]  

...

Epoch [99900/100000]
Train Loss: 5.8850, Train Accuracy: 0.0855
Train Precision (micro): 0.0855, Train Precision (macro): 0.0093
Train Recall (micro): 0.0855, Train Recall (macro): 0.0136
Train F1 (micro): 0.0855, Train F1 (macro): 0.0087

Val Loss: 5.9408, Val Accuracy: 0.0744
Val Precision (micro): 0.0744, Val Precision (macro): 0.0083
Val Recall (micro): 0.0744, Val Recall (macro): 0.0130
Val F1 (micro): 0.0744, Val F1 (macro): 0.0074
--------------------------------------------------------------------------------


100%|█████████▉| 99999/100000 [58:52<00:00, 48.83it/s]

Epoch [100000/100000]
Train Loss: 5.9066, Train Accuracy: 0.0786
Train Precision (micro): 0.0786, Train Precision (macro): 0.0052
Train Recall (micro): 0.0786, Train Recall (macro): 0.0096
Train F1 (micro): 0.0786, Train F1 (macro): 0.0055

Val Loss: 5.9248, Val Accuracy: 0.0766
Val Precision (micro): 0.0766, Val Precision (macro): 0.0079
Val Recall (micro): 0.0766, Val Recall (macro): 0.0134
Val F1 (micro): 0.0766, Val F1 (macro): 0.0080
--------------------------------------------------------------------------------


100%|██████████| 100000/100000 [58:53<00:00, 28.30it/s]


epoch,▁▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▆▆▆▆▆▆▆▇▇▇██
lr,▂▅▁▃█▁████▆▅▄▄▃▂▂▁███▇▇▇▇▆▅▅▅▄▂▂▁▁█████▇
train_accuracy,▁▃▄▆▅▅▄▆▅▆▆▅▃▅▅▆▆▇██▆▆▆▅▇▇▆▆▅▆▆▇▆██▇▆▅█▇
train_f1_macro,▁▂▃▃▃▅▆▅▅▅▆▆▇▅▆▆▆▇▅▆▆▇▅▆▆█▆▆▇▇▇▆▇▆▆▇▇▆▇▆
train_f1_micro,▁▄▆▆█▇▇█▇▇█▇█▇█▇▇▇█▇█▇▇▆█▇███▇███▇██▇▇██
train_loss,█▄▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▂▂▁▂▂▁▂▁▁▁▁▁▂▁▂▂▂▂▂▂
train_precision_macro,▁▂▃▄▅▇▃▄▆▄▄▇▅▅▆▄▄▅▃▄▇▄▅▄▆▆▇▆▇▇▅█▆▆▆▇▅▅██
train_precision_micro,▁▃▄▅▇▇▆▇▆▇▇▇▇▇▆▆▇█▇█▇▇▆▆▇▇▇█▇▇▇▇▇▇▇▇▇▇▆▆
train_recall_macro,▁▃▃▄▄▅▆▆▅▆▅▅▆▅▆▅▆▅▅▆▆▇▅▆▆▆▆▆▇▆▇▆▆█▇▆▆▅▅▇
train_recall_micro,▁▆▅▄▅▇▆▇▆▇▆▇█▇▆▇█▆▇▇▇▇▆▆▆▆▇▇█▆▇█▇▇▇▆▇▇▇█
val_accuracy,▁▃▂▄▅▆▅▆▅▇▇▆▆█▇▇▇█▇▇▆▆▆▆▆▇█▆▇▆▇▇▆▇▆███▆▆


In [28]:
input_seq = "TLWQRPLVSIKVGGQIKEALLDTGADDTVLEDINLPGKWKPKMIGGIGGFIKVKQYDNITKILDPFRAKNPELVIYQYMDDLYVGSDLEIGQHRAKIEE"
true_tax_id = 12721
Label = map_label[true_tax_id]
Label

2773

In [29]:
from taxonkit_util.py import trace_back

trace_back(str(Label))

ModuleNotFoundError: No module named 'taxonkit_util.py'; 'taxonkit_util' is not a package

In [26]:
[map_label[e['Taxonomic_lineage_IDs']] for e in b]

input_tensor = torch.LongTensor([encode_sequence("")])
input_tensor = input_tensor.to(device)
outputs = model(input_tensor)
print(outputs)
preds = torch.argmax(outputs, dim=1).item()
idx_to_label[preds]

tensor([[ -4.1667, -18.9106,  -1.9891,  ..., -15.5333, -11.1934, -17.2228]],
       device='cuda:1', grad_fn=<AddmmBackward0>)


327045

In [ ]:
"Orthoretrovirinae"

In [ ]:
probs = torch.nn.functional.softmax(outputs, dim=1)
# Get top k probabilities and indices
top_probs, top_indices = torch.topk(probs, k, dim=1)